# Week 7 Homework

***Due (pushed to your GitHub branch) on 10/31 by 11:59 pm***

## Samtools

Use Samtools to convert to BAM and sort the output SAMs produced by HISAT:

In [1]:
!for sam in /data/users/zhouz6436/BIOL343/6_alignment/hisat/*.sam; do \
    bn=$(basename "$sam" .sam); \
    echo "Converting $sam to $bn.bam and sorting."; \
    samtools view -@ 32 -b "$sam" | samtools sort -@ 32 -o /data/users/zhouz6436/BIOL343/7_alignment_qc/${bn}_sorted.bam; \
done

Converting /data/users/zhouz6436/BIOL343/6_alignment/hisat/SRR2916027.sam to SRR2916027.bam and sorting.
[bam_sort_core] merging from 0 files and 32 in-memory blocks...
Converting /data/users/zhouz6436/BIOL343/6_alignment/hisat/SRR2916028.sam to SRR2916028.bam and sorting.
[bam_sort_core] merging from 0 files and 32 in-memory blocks...
Converting /data/users/zhouz6436/BIOL343/6_alignment/hisat/SRR2916029.sam to SRR2916029.bam and sorting.
[bam_sort_core] merging from 0 files and 32 in-memory blocks...
Converting /data/users/zhouz6436/BIOL343/6_alignment/hisat/SRR2916030.sam to SRR2916030.bam and sorting.
[bam_sort_core] merging from 0 files and 32 in-memory blocks...
Converting /data/users/zhouz6436/BIOL343/6_alignment/hisat/SRR2916031.sam to SRR2916031.bam and sorting.
[bam_sort_core] merging from 0 files and 32 in-memory blocks...
Converting /data/users/zhouz6436/BIOL343/6_alignment/hisat/SRR2916032.sam to SRR2916032.bam and sorting.
[bam_sort_core] merging from 0 files and 32 in-mem

Merge the BAMs to a single BAM and generate an index:

In [6]:
%cd /data/users/zhouz6436/BIOL343/7_alignment_qc
!samtools merge -@ 32 merged_alignment.bam *.bam
!samtools index -@ 32 merged_alignment.bam


/data/users/zhouz6436/BIOL343/7_alignment_qc


Use Samtools `flagstat` to generate QC metrics for the STAR and HISAT merged BAMs:

In [1]:
!samtools flagstat /data/users/zhouz6436/BIOL343/7_alignment_qc/merged_alignment.bam

1001500375 + 0 in total (QC-passed reads + QC-failed reads)
112025645 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
871182746 + 0 mapped (86.99% : N/A)
0 + 0 paired in sequencing
0 + 0 read1
0 + 0 read2
0 + 0 properly paired (N/A : N/A)
0 + 0 with itself and mate mapped
0 + 0 singletons (N/A : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


## Qualimap

Use Qualimap to run `bamqc` and `rnaseq` to generate QC metrics for the STAR and HISAT merged BAMs:

In [1]:
!qualimap bamqc -nt 32 --java-mem-size=32G -outdir qualimap/star/bam -bam /data/users/zhouz6436/BIOL343/6_alignment/alignment/star_for_homework/second_pass_Aligned.fixed.sorted.bam
!qualimap rnaseq --java-mem-size=32G -outdir qualimap/star/rnaseq -bam /data/users/zhouz6436/BIOL343/6_alignment/alignment/star_for_homework/second_pass_Aligned.fixed.sorted.bam -gtf /data/users/zhouz6436/BIOL343/2_genome_exploration/genome_for_homework/annotations_for_homework.gtf
!qualimap bamqc -nt 32 --java-mem-size=32G -outdir qualimap/hisat/bam -bam /data/users/zhouz6436/BIOL343/7_alignment_qc/merged_alignment.bam --feature-file ../2_genome_exploration/genome_for_homework/annotations_for_homework.gtf
!qualimap rnaseq --java-mem-size=32G -outdir qualimap/hisat/rnaseq -bam /data/users/zhouz6436/BIOL343/7_alignment_qc/merged_alignment.bam -gtf ../2_genome_exploration/genome_for_homework/annotations_for_homework.gtf 

Java memory size is set to 32G
Launching application...

[0.002s][warning][os,container] Duplicate cpuset controllers detected. Picking /sys/fs/cgroup/cpuset, skipping /dev/cpuset.
QualiMap v.2.3
Built on 2023-05-19 16:57

Selected tool: bamqc
Available memory (Mb): 33
Max memory (Mb): 34359
Starting bam qc....
Loading sam header...
Loading locator...
Loading reference...
Number of windows: 400, effective number of windows: 460
Chunk of reads size: 1000
Number of threads: 32
Processed 50 out of 460 windows...
Processed 100 out of 460 windows...
Processed 150 out of 460 windows...
Processed 200 out of 460 windows...
Processed 250 out of 460 windows...
Processed 300 out of 460 windows...
Processed 350 out of 460 windows...
Processed 400 out of 460 windows...
Processed 450 out of 460 windows...
Total processed windows:460
Number of reads: 899649259
Number of valid reads: 830766740
Number of correct strand reads:0

Inside of regions...
Num mapped reads: 830766740
Num mapped first of pair: 

In [3]:
%cd /data/users/zhouz6436/BIOL343/6_alignment/alignment/star_for_homework
!samtools view -H second_pass_Aligned.out.bam > header.sam
!sed -i 's/^@RG\tID:\(.*\)/@RG\tID:\1\tSM:\1/' header.sam
!samtools reheader header.sam second_pass_Aligned.out.bam > second_pass_Aligned.fixed.bam
!samtools index second_pass_Aligned.fixed.bam


/data/users/zhouz6436/BIOL343/6_alignment/alignment/star_for_homework
[E::hts_idx_push] Chromosome blocks not continuous
samtools index: failed to create index for "second_pass_Aligned.fixed.bam"


In [4]:
!samtools sort -@ 16 \
  -o second_pass_Aligned.fixed.sorted.bam \
  second_pass_Aligned.fixed.bam
!samtools index second_pass_Aligned.fixed.sorted.bam

[bam_sort_core] merging from 352 files and 16 in-memory blocks...


## Report

Use MuliQC to generate a single report that merges all Samtools and Qualimap metrics:

In [8]:
!multiqc --force -d ../5_fastq/mouse_hippocampus_fastq/qc/ ../6_alignment/trimmed/ ../7_alignment_qc/qualimap .




  /// ]8;id=287686;https://multiqc.info\MultiQC]8;;\ 🔍 | v1.17

|           multiqc | Prepending directory to sample names
|           multiqc | Search path : /data/users/zhouz6436/BIOL343/5_fastq/mouse_hippocampus_fastq/qc
|           multiqc | Search path : /data/users/zhouz6436/BIOL343/6_alignment/trimmed
|           multiqc | Search path : /data/users/zhouz6436/BIOL343/7_alignment_qc/qualimap
|           multiqc | Search path : /data/users/zhouz6436/BIOL343/7_alignment_qc
|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 420/420  at/rnaseq/css/doctools.jsry.js
|          qualimap | Found 4 BamQC reports
|          qualimap | Found 4 RNASeq reports
|          samtools | Found 1 stats reports
|          samtools | Found 2 flagstat reports
|            fastqc | Found 23 reports
|           multiqc | Report      : multiqc_report.html
|           multiqc | Data        : multiqc_data
|           multiqc | MultiQC complete


## JBrowse2

Upload a screenshot of JBrowse2 that shows the reference genome, the annotations, and the two alignments as tracks.

## Snakemake Pipeline (Optional)

1. Copy your `Snakefile` and `config.yaml` from `6_alignment` to `7_alignment_qc`.  
2. In the `Snakefile`, add the following rules:  
    a. `convert_hisat` - use `samtools` to convert HISAT SAMs to BAMs; sort the BAMs.  
    b. `merge_hisat` - use `samtools` to merge all HISAT BAMs.  
    c. `flagstat` -  use `samtools` to generate `flagstat` QC.  
    d. `qualimap` - use `qualimap` to generate `bamqc` and `rnaseq` QC.  
    e. `report` - use `multiqc` to aggregate all QC data into a single report. This should be an update of the report from the previous week's homework.  

This `Snakefile` should not use any `wrappers` because we already have the relevant software installed in the `biol343` conda environment. The pipeline should run to completion when any instructor or classmate runs `snakemake --use-conda`.